In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time
import re
from bs4 import BeautifulSoup
from tqdm import tqdm
import urllib
from urllib.request import urlopen

In [5]:
df = pd.read_csv('./서울시 착한가격음식점(종합).csv',encoding='cp949',index_col=0)
df

,업소아이디,업소명,분류코드,분류코드명,업소 주소,업소 전화번호,찾아오시는 길,업소정보,자랑거리,추천수,...,키워드1,개수1,키워드2,개수2,키워드3,개수3,키워드4,개수4,키워드5,개수5
0,1739,목우촌부추삼겹살,1,한식,서울특별시 관악구 남현1길 68-10 (남현동),588-7399,전철 2호선 4호선 사당역에서 5번출구로 나오시면 (구)관악등기소쪽 먹자골목으로 5...,영업시간 : 오전 10시 ~ 오후 11시 30분 좌석수 : 44석 휴무일 : 연중무...,▷사당역 먹자골목에 위치 ▷김치찌개된장찌개 2010년부터 가격 동결 ▷친절업소,257,...,음식이 맛있어요,47,가성비가 좋아요,21,재료가 신선해요,15,친절해요,12,양이 많아요,8
1,4325,청진동해장국,1,한식,서울특별시 관악구 관악로 105 (봉천동) 동산빌딩1층,875-9882,지하철 2호선 서울대입구역 3번 출구로 나와서 서울대학교 쪽으로 직진 도보10분 관...,영업시간 : 24시간 좌석수 : 80석 휴무일 : 연중무휴 배달 : 불가 예약 :...,관악구 평균가격(5000원) 보다 10% 저렴 김치 깍두기 등 직접 만들어 신선함.,252,...,0,0,0,0,0,0,0,0,0,0
2,7851,오백냥분식,1,한식,서울특별시 송파구 가락로 112 (석촌동),414-2261,전철노선: 8호선 송파역 4번출구 중동빌딩 버스노서 : 3417.3418.3422....,영업시간:08:00~21:00 휴무일:연중무휴 좌 석:20석 주 차:가능,배 달:불가 예 약:불가 카 드:가능,236,...,가성비가 좋아요,32,음식이 맛있어요,26,혼밥하기 좋아요,25,친절해요,16,양이 많아요,10
3,6764,토방 닭 한마리,1,한식,서울특별시 관악구 신림로22길 5 (신림동),874-6895,신성부동산 건물 바로 뒷건물,영업시간 : 오전 10시 ~ 오후 10시 좌석수 : 50석 예약 : 가능 카드 : ...,▷관악구 평균가격보다 10% 저렴 ▷반찬을 여러가지 하지 않고 김치와 깍두기 두가지...,214,...,음식이 맛있어요,83,친절해요,32,양이 많아요,24,가성비가 좋아요,24,재료가 신선해요,19
4,3986,왕세숫대야냉면(행복을파는집),1,한식,서울특별시 강서구 공항대로 423(등촌동),02-2668-3608,화진코스메틱에서 강서구청4거리쪽으로 조금만 올라오세요~,영업시간 : 24시간 좌석수 : 30개 휴무일 : 미정 배달 : 불가 카드 : 가능...,많이 판매가 되므로 저렴한 가격유지 그날그날 신선한 재료사용하여 조리합니다.,163,...,음식이 맛있어요,732,가성비가 좋아요,654,양이 많아요,381,혼밥하기 좋아요,283,친절해요,141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728,9156,삼겹살데이,1,한식,서울특별시 서대문구 가좌로 29 (연희동),324-0301,NaN,영업시간 : 좌석수 : 휴무일 : 배달 : 예약 : 카드 : 주차 : 홈페이지 주소...,NaN,0,...,음식이 맛있어요,60,가성비가 좋아요,33,친절해요,25,양이 많아요,14,재료가 신선해요,13
729,9101,홍능갈비,1,한식,서울특별시 금천구 범안로21길 6 (독산동),02-894-8292,NaN,영업시간 : 12:00~23:00 좌석수 : 휴무일 : 배달 : 불가 예약 : 카드...,NaN,0,...,0,0,0,0,0,0,0,0,0,0
730,9315,보물쌈,1,한식,서울특별시 동작구 상도로 296-2 (상도1동),814-2151,NaN,저렴한 가격 깨끗한 위생,저렴한 가격 깨끗한 위생,0,...,음식이 맛있어요,71,가성비가 좋아요,37,양이 많아요,25,친절해요,21,혼밥하기 좋아요,11
731,9279,가정식당,1,한식,서울특별시 강남구 학동로77길 20 (청담동),02-514-8009,NaN,영업시간 :09:00~21:00 좌석수 : 휴무일 : 배달 :가능 예약 : 카드 :...,NaN,0,...,음식이 맛있어요,18,가성비가 좋아요,13,친절해요,11,반찬이 잘 나와요,8,재료가 신선해요,6


In [23]:
#https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EB%AA%A9%EC%9A%B0%EC%B4%8C%EB%B6%80%EC%B6%94%EC%82%BC%EA%B2%B9%EC%82%B4
base_url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query='
place = review['행정구'].iloc[0]+review['행정동'].iloc[0]+review['업소명'].iloc[0]
url = base_url + urllib.parse.quote(place)
url

'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EA%B4%80%EC%95%85%EA%B5%AC%EB%82%A8%ED%98%84%EB%8F%99%EB%AA%A9%EC%9A%B0%EC%B4%8C%EB%B6%80%EC%B6%94%EC%82%BC%EA%B2%B9%EC%82%B4'

In [24]:
driver = webdriver.Chrome('./driver/chromedriver')
driver.get(url)

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_1988/766079133.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./driver/chromedriver')


In [29]:
search = driver.find_element(By.ID,'query')
search.clear()
search.send_keys(place)
xpath = '//*[@id="search_btn"]'
driver.find_element(By.XPATH,xpath).click()
xpath2 = '//*[@id="place-main-section-root"]/section/div/div[3]/div/a'
driver.find_element(By.XPATH,xpath2).click()

html = urlopen(driver.current_url)
bs_obj = bs4.BeautifulSoup(html,'html.parser')
rev = bs_obj.find_all('span',{'class':'_10uv0'})
rev_num = bs_obj.find_all('span',{'class':'_3IFxe'})
driver.close()
print(place)

NameError: name 'By' is not defined

In [22]:
for i in range(len(review)-1) :
    base_url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query='
    place = review['행정구'].iloc[i]+review['행정동'].iloc[i]+review['업소명'].iloc[i]
    url = base_url + urllib.parse.quote(place)
    driver = webdriver.Chrome('./driver/chromedriver')


0 https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EA%B4%80%EC%95%85%EA%B5%AC%EB%82%A8%ED%98%84%EB%8F%99%EB%AA%A9%EC%9A%B0%EC%B4%8C%EB%B6%80%EC%B6%94%EC%82%BC%EA%B2%B9%EC%82%B4
1 https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EA%B4%80%EC%95%85%EA%B5%AC%EB%B4%89%EC%B2%9C%EB%8F%99%EC%B2%AD%EC%A7%84%EB%8F%99%ED%95%B4%EC%9E%A5%EA%B5%AD
2 https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EC%86%A1%ED%8C%8C%EA%B5%AC%EC%84%9D%EC%B4%8C%EB%8F%99%EC%98%A4%EB%B0%B1%EB%83%A5%EB%B6%84%EC%8B%9D
3 https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EA%B4%80%EC%95%85%EA%B5%AC%EC%8B%A0%EB%A6%BC%EB%8F%99%ED%86%A0%EB%B0%A9%20%EB%8B%AD%20%ED%95%9C%EB%A7%88%EB%A6%AC
4 https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EA%B0%95%EC%84%9C%EA%B5%AC%EB%93%B1%EC%B4%8C%EB%8F%99%EC%99%95%EC%84%B8%EC%88%AB%EB%8C%80%EC%95%BC%EB%83%89%EB%A9%B4%2

519 https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EC%84%9C%EB%8C%80%EB%AC%B8%EA%B5%AC%EC%B0%BD%EC%B2%9C%EB%8F%99%EB%8C%80%EB%AA%85%EA%BC%AC%EA%B8%B0
520 https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EC%84%9C%EB%8C%80%EB%AC%B8%EA%B5%AC%EB%B6%81%EA%B0%80%EC%A2%8C%EB%8F%99%EC%86%8C%EB%AC%B8%EB%82%9C%EB%83%89%EB%A9%B4
521 https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EC%84%9C%EB%8C%80%EB%AC%B8%EA%B5%AC%ED%99%8D%EC%A0%9C%EB%8F%99%EC%B9%BC%EC%A7%91%EC%82%BC%EA%B2%B9%EC%82%B4
522 https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EC%84%9C%EB%8C%80%EB%AC%B8%EA%B5%AC%ED%99%8D%EC%A0%9C%EB%8F%99%ED%99%8D%EC%A0%9C%EB%8F%99%EC%9E%94%EC%B9%98%EC%A7%91
523 https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EA%B0%95%EB%82%A8%EA%B5%AC%EB%8F%84%EA%B3%A1%EB%8F%99%EC%9B%90%ED%95%A0%EB%A7%A4%EC%9D%B4%EB%AA%A8%EB%84%A4%EB%8B%

671 https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EA%B4%80%EC%95%85%EA%B5%AC%EC%8B%A0%EB%A6%BC%EB%8F%99%EC%88%98%EC%9E%90%ED%83%80%EC%B0%A9%ED%95%9C%EC%8B%9D%EB%8B%B9
672 https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EC%A4%91%EB%9E%91%EA%B5%AC%EB%A9%B4%EB%AA%A9%EB%8F%99%EB%8F%99%ED%95%B4%EB%A3%A8
673 https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EC%A4%91%EB%9E%91%EA%B5%AC%EB%A9%B4%EB%AA%A9%EB%8F%99%EC%A7%9C%EC%9E%A5%EB%82%98%EB%A3%A8
674 https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EC%A4%91%EB%9E%91%EA%B5%AC%EC%A4%91%ED%99%94%EB%8F%99%EC%BB%A4%ED%94%BC%EC%95%A4%EB%B0%94%EB%B3%B5%EA%B8%B8%EC%9D%B4
675 https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EA%B4%80%EC%95%85%EA%B5%AC%EC%8B%A0%EB%A6%BC%EB%8F%99%EA%B5%BF%EB%9D%B5%EC%BB%A4%ED%94%BC
676 https://search.naver.com/search.naver?where=nexearch&sm=

In [204]:
review = review[['행정구','행정동']]
review

,위치한 구,업소명
0,도봉구,우이목장
1,도봉구,착한부대찌개
2,도봉구,홍방원
3,중랑구,오늘우리집비어
4,은평구,은평닭곰탕
...,...,...
696,강북구,원조막국수
697,송파구,해주냉면
698,용산구,감베로니
699,송파구,장수설렁탕


In [234]:
driver = webdriver.Chrome('./driver/chromedriver')

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_21556/4139024571.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./driver/chromedriver')


In [235]:
base_url = 'https://www.naver.com/'
driver.get(base_url)

In [236]:
search = driver.find_element(By.ID,'query')

In [237]:
search.clear()
search.send_keys(a)

In [238]:
xpath = '//*[@id="search_btn"]'
driver.find_element(By.XPATH,xpath).click()

In [239]:
xpath2 = '//*[@id="place-main-section-root"]/section/div/div[3]/div/a'
driver.find_element(By.XPATH,xpath2).click()

In [ ]:
#https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=%EB%8F%84%EB%B4%89%EA%B5%AC+%EC%9A%B0%EC%9D%B4%EB%AA%A9%EC%9E%A5

In [240]:
a = review['위치한 구'].iloc[0],'',review['업소명'].iloc[0]
a

('도봉구', '', '우이목장')

In [266]:
review.info

<bound method DataFrame.info of     위치한 구      업소명
0     도봉구     우이목장
1     도봉구   착한부대찌개
2     도봉구      홍방원
3     중랑구  오늘우리집비어
4     은평구    은평닭곰탕
..    ...      ...
696   강북구    원조막국수
697   송파구     해주냉면
698   용산구     감베로니
699   송파구    장수설렁탕
700   용산구      무진장

[701 rows x 2 columns]>

In [ ]:
for i in range(len(review)-1) : 
    try :
        driver = webdriver.Chrome('./driver/chromedriver')
        base_url = 'https://www.naver.com/'
        driver.get(base_url)
        place = review['위치한 구'].iloc[i],review['업소명'].iloc[i]
        search = driver.find_element(By.ID,'query')
        search.clear()
        search.send_keys(place)
        xpath = '//*[@id="search_btn"]'
        driver.find_element(By.XPATH,xpath).click()
        xpath2 = '//*[@id="place-main-section-root"]/section/div/div[3]/div/a'
        driver.find_element(By.XPATH,xpath2).click()

        html = urlopen(driver.current_url)
        bs_obj = bs4.BeautifulSoup(html,'html.parser')
        rev = bs_obj.find_all('span',{'class':'_10uv0'})
        rev_num = bs_obj.find_all('span',{'class':'_3IFxe'})
        driver.close()
        print(place)
        for i in range(len(rev)) :
            print(rev[i].text, re.sub(r'[^0-9]','',rev_num[i].text))
    
    except :
        print("오류")


C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_21556/235786355.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./driver/chromedriver')


('도봉구', '우이목장')
"음식이 맛있어요" 48
"가성비가 좋아요" 16
"친절해요" 12
"재료가 신선해요" 17
"매장이 넓어요" 14
('도봉구', '착한부대찌개')
"음식이 맛있어요" 33
"친절해요" 12
"재료가 신선해요" 6
"가성비가 좋아요" 21
"양이 많아요" 8
오류
('중랑구', '오늘우리집비어')
"음식이 맛있어요" 16
"가성비가 좋아요" 11
"기본 안주가 좋아요" 7
"친절해요" 12
"매장이 청결해요" 9


Exception ignored in: <function Service.__del__ at 0x000001C7D838ADC0>
Traceback (most recent call last):
  File "C:\Users\Administrator\anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 171, in __del__
    self.stop()
  File "C:\Users\Administrator\anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 146, in stop
    self.send_remote_shutdown_command()
  File "C:\Users\Administrator\anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 122, in send_remote_shutdown_command
    request.urlopen("%s/shutdown" % self.service_url)
  File "C:\Users\Administrator\anaconda3\lib\urllib\request.py", line 214, in urlopen
    return opener.open(url, data, timeout)
  File "C:\Users\Administrator\anaconda3\lib\urllib\request.py", line 517, in open
    response = self._open(req, data)
  File "C:\Users\Administrator\anaconda3\lib\urllib\request.py", line 534, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File 

오류
오류
오류
오류
오류


Exception ignored in: <function Service.__del__ at 0x000001C7D838ADC0>
Traceback (most recent call last):
  File "C:\Users\Administrator\anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 171, in __del__
    self.stop()
  File "C:\Users\Administrator\anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 146, in stop
    self.send_remote_shutdown_command()
  File "C:\Users\Administrator\anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 127, in send_remote_shutdown_command
    if not self.is_connectable():
  File "C:\Users\Administrator\anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 116, in is_connectable
    return utils.is_connectable(self.port)
  File "C:\Users\Administrator\anaconda3\lib\site-packages\selenium\webdriver\common\utils.py", line 107, in is_connectable
    socket_ = socket.create_connection((host, port), 1)
  File "C:\Users\Administrator\anaconda3\lib\socket.py", line 832, in create_con

('은평구', '우경촌')
"음식이 맛있어요" 23
"재료가 신선해요" 6
"친절해요" 3
"가성비가 좋아요" 12
"양이 많아요" 5
오류
('은평구', 'OK스시')
오류


Exception ignored in: <function Service.__del__ at 0x000001C7D838ADC0>
Traceback (most recent call last):
  File "C:\Users\Administrator\anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 171, in __del__
    self.stop()
  File "C:\Users\Administrator\anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 146, in stop
    self.send_remote_shutdown_command()
  File "C:\Users\Administrator\anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 122, in send_remote_shutdown_command
    request.urlopen("%s/shutdown" % self.service_url)
  File "C:\Users\Administrator\anaconda3\lib\urllib\request.py", line 214, in urlopen
    return opener.open(url, data, timeout)
  File "C:\Users\Administrator\anaconda3\lib\urllib\request.py", line 517, in open
    response = self._open(req, data)
  File "C:\Users\Administrator\anaconda3\lib\urllib\request.py", line 534, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File 

오류
오류
('도봉구', '카페매일이기념일')
"커피가 맛있어요" 15
"친절해요" 12
"음료가 맛있어요" 7
"디저트가 맛있어요" 14
"매장이 청결해요" 9
오류
오류
오류
오류
('구로구', '윤경이네')
오류
오류
오류
오류


In [262]:
url = f'https://map.naver.com/v5/search/{place_p}/place/'
place = review['위치한 구'].iloc[0]+' '+ review['업소명'].iloc[0]
place_p = urllib.parse.quote(place)

driver = webdriver.Chrome('./driver/chromedriver')
driver.get(url)

# https://map.naver.com/v5/search/%EB%8F%84%EB%B4%89%EA%B5%AC%20%EC%9A%B0%EC%9D%B4%EB%AA%A9%EC%9E%A5/place
# https://map.naver.com/v5/search/%EB%8F%84%EB%B4%89%EA%B5%AC%20%EC%9A%B0%EC%9D%B4%EB%AA%A9%EC%9E%A5/place/38431899?c=14140389.8214314,4529301.4884308,15,0,0,0,dh&isCorrectAnswer=true

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_21556/786722113.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./driver/chromedriver')


In [ ]:
'https://map.naver.com/v5/entry/place/21025096?c=14135011.4202338,4505580.5594894,15,0,0,0,dh'

In [180]:
df = pd.read_csv('./addrs.csv')
driver = webdriver.Chrome('./driver/chromedriver')

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_21556/2252704643.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./driver/chromedriver')


In [179]:
from urllib.request import urlopen
import bs4
import pandas as pd
# base_url = f'https://pcmap.place.naver.com/restaurant/{keyword}/review/visitor#'
url = 'https://pcmap.place.naver.com/restaurant/21025096/review/visitor#'
driver.get(url)

WebDriverException: Message: chrome not reachable
  (Session info: chrome=104.0.5112.81)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B45FD3+2187219]
	Ordinal0 [0x00ADE6D1+1763025]
	Ordinal0 [0x009F3D40+802112]
	Ordinal0 [0x009E8A12+756242]
	Ordinal0 [0x009E9248+758344]
	Ordinal0 [0x009EAB02+764674]
	Ordinal0 [0x009E4379+738169]
	Ordinal0 [0x009F51D0+807376]
	Ordinal0 [0x00A4C7A2+1165218]
	Ordinal0 [0x00A3C9A6+1100198]
	Ordinal0 [0x00A16F80+946048]
	Ordinal0 [0x00A17E76+949878]
	GetHandleVerifier [0x00DE90C2+2721218]
	GetHandleVerifier [0x00DDAAF0+2662384]
	GetHandleVerifier [0x00BD137A+526458]
	GetHandleVerifier [0x00BD0416+522518]
	Ordinal0 [0x00AE4EAB+1789611]
	Ordinal0 [0x00AE97A8+1808296]
	Ordinal0 [0x00AE9895+1808533]
	Ordinal0 [0x00AF26C1+1844929]
	BaseThreadInitThunk [0x7728FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77897A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77897A6E+238]


In [106]:
xpath = '//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[1]/div/div/div[2]/a'
driver.find_element(By.XPATH,xpath).click()

In [110]:
html = urlopen(url)
bs_obj = bs4.BeautifulSoup(html,'html.parser')

In [141]:
rev = bs_obj.find_all('span',{'class':'_10uv0'})
rev

[<span class="_10uv0">"<!-- -->음식이 맛있어요<!-- -->"</span>,
 <span class="_10uv0">"<!-- -->가성비가 좋아요<!-- -->"</span>,
 <span class="_10uv0">"<!-- -->재료가 신선해요<!-- -->"</span>,
 <span class="_10uv0">"<!-- -->친절해요<!-- -->"</span>,
 <span class="_10uv0">"<!-- -->양이 많아요<!-- -->"</span>]

In [155]:
rev_num = bs_obj.find_all('span',{'class':'_3IFxe'})
rev_num

[<span class="_3IFxe"><span class="place_blind">이 키워드를 선택한 인원</span>47</span>,
 <span class="_3IFxe"><span class="place_blind">이 키워드를 선택한 인원</span>21</span>,
 <span class="_3IFxe"><span class="place_blind">이 키워드를 선택한 인원</span>15</span>,
 <span class="_3IFxe"><span class="place_blind">이 키워드를 선택한 인원</span>12</span>,
 <span class="_3IFxe"><span class="place_blind">이 키워드를 선택한 인원</span>8</span>]

In [158]:
import re
for i in range(len(rev)) :
    print
    print(rev[i].text, re.sub(r'[^0-9]','',rev_num[i].text))

"음식이 맛있어요" 47
"가성비가 좋아요" 21
"재료가 신선해요" 15
"친절해요" 12
"양이 많아요" 8


In [162]:
df.head()

,Unnamed: 0,업소명,업소 주소
0,0,목우촌부추삼겹살,서울특별시 관악구 남현1길 68-10 (남현동)
1,1,청진동해장국,서울특별시 관악구 관악로 105 (봉천동) 동산빌딩1층
2,2,오백냥분식,서울특별시 송파구 가락로 112 (석촌동)
3,3,토방 닭 한마리,서울특별시 관악구 신림로22길 5 (신림동)
4,4,왕세숫대야냉면(행복을파는집),서울특별시 강서구 공항대로 423(등촌동)


In [181]:
#for i, keyword in enumerate(df['업소명'].tolist()) :
import urllib
keyword = '남현동 목우촌부추삼겹살'

naver_map_url = f"https://pcmap.place.naver.com/restaurant/{keyword}/review/visitor#"
naver_map_url
driver.get(naver_map_url)

In [97]:
for i, keyword in enumerate(df['업소명'].tolist()): 
    print("이번에 찾을 키워드 :", i, f"/ {df.shape[0]} 행", keyword) 
    try:
        naver_map_search_url = f'https://map.naver.com/v5/search/{keyword}/place'
        # 검색 url 만들기
        driver.get(naver_map_search_url)
        # 검색 url 접속, 즉 검색하기
        time.sleep(4) 
        cu = driver.current_url # 검색이 성공된 플레이스에 대한 개별 페이지
        res_code = re.findall(r"place/(\d+)", cu)
        final_url = 'https://pcmap.place.naver.com/restaurant/'+res_code[0]+'/review/visitor#'
        print(final_url)
        df['naver_map_url'][i]=final_url
    except IndexError:
        df['naver_map_url'][i]= ''
        print('none')
        df.to_csv('url_completed.csv', encoding = 'utf-8')

이번에 찾을 키워드 : 0 / 734 행 목우촌부추삼겹살
https://pcmap.place.naver.com/restaurant/21025096/review/visitor#
이번에 찾을 키워드 : 1 / 734 행 청진동해장국


C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_21556/500988348.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['naver_map_url'][i]=final_url
C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_21556/500988348.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['naver_map_url'][i]= ''


none
이번에 찾을 키워드 : 2 / 734 행 오백냥분식
none
이번에 찾을 키워드 : 3 / 734 행 토방 닭 한마리
none
이번에 찾을 키워드 : 4 / 734 행 왕세숫대야냉면(행복을파는집)
none
이번에 찾을 키워드 : 5 / 734 행 돌마리유황오리
none
이번에 찾을 키워드 : 6 / 734 행 벧엘 칼국수
none
이번에 찾을 키워드 : 7 / 734 행 보라매25시해장국
none
이번에 찾을 키워드 : 8 / 734 행 해피분식
none
이번에 찾을 키워드 : 9 / 734 행 금강숯불생고기
none
이번에 찾을 키워드 : 10 / 734 행 흥부농장
none
이번에 찾을 키워드 : 11 / 734 행 한우마당
none
이번에 찾을 키워드 : 12 / 734 행 김밥나라
none
이번에 찾을 키워드 : 13 / 734 행 다비다식당
none
이번에 찾을 키워드 : 14 / 734 행 자매식당
none
이번에 찾을 키워드 : 15 / 734 행 궁원 중화요리
none
이번에 찾을 키워드 : 16 / 734 행 제주토방탕수육


KeyboardInterrupt: 

In [98]:
df

,Unnamed: 0,업소명,업소 주소,naver_map_url
0,0,목우촌부추삼겹살,서울특별시 관악구 남현1길 68-10 (남현동),https://pcmap.place.naver.com/restaurant/21025...
1,1,청진동해장국,서울특별시 관악구 관악로 105 (봉천동) 동산빌딩1층,
2,2,오백냥분식,서울특별시 송파구 가락로 112 (석촌동),
3,3,토방 닭 한마리,서울특별시 관악구 신림로22길 5 (신림동),
4,4,왕세숫대야냉면(행복을파는집),서울특별시 강서구 공항대로 423(등촌동),
...,...,...,...,...
729,729,삼겹살데이,서울특별시 서대문구 가좌로 29 (연희동),
730,730,홍능갈비,서울특별시 금천구 범안로21길 6 (독산동),
731,731,보물쌈,서울특별시 동작구 상도로 296-2 (상도1동),
732,732,가정식당,서울특별시 강남구 학동로77길 20 (청담동),


In [ ]:
df = pd.read_csv('url_completed.csv') #null값 처리 완료 데이터
# 수집할 정보들
rating_list = []
# 평점
user_review_id = {}
# 유저의 id - 추천시스템에서는 필수
review_json = {}
# 리뷰
image_json = {} # 이미지

In [ ]:
for i in range(len(df)):
    print('======================================================')
    print(str(i)+'번째 식당') # 식당 리뷰 개별 url 접속
    driver.get(df['naver_map_url'][i])
    thisurl = df['naver_map_url'][i] time.sleep(2)
    # 더보기 버튼 다 누르기 
    # 더보기 버튼 누르기까지 10개
    # 더보기 버튼 누르면 10개 추가됨
    while True:
        try: time.sleep(1)
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(3)
            driver.find_element_by_css_selector('#app-root > div > div.place_detail_wrapper > div:nth-child(5) > div:nth-child(4) > div:nth-child(4) > div._2kAri > a').click()
            time.sleep(3)
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(1)
        except NoSuchElementException:
            print('-더보기 버튼 모두 클릭 완료-')
        break # 파싱 
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        time.sleep(1) # 식당 구분
        restaurant_name = df['검색어'][i]
        print('식당 이름 : '+restaurant_name)
        user_review_id[restaurant_name] = {}
        review_json[restaurant_name] = {}
        image_json[restaurant_name] = {} 
        try: 
            restaurant_classificaton = soup.find_all('span',attrs = {'class':'_3ocDE'})[0].text
            except: restaurant_classificaton = 'none'
                print('식당 구분 : '+restaurant_classificaton)
                print('----------------------------------------------') 
                try: one_review = soup.find_all('div', attrs = {'class':'_1Z_GL'})
                    review_num = len(one_review)
                    # 특정 식당의 리뷰 총 개수 
                    print('리뷰 총 개수 : '+str(review_num)) 
                    # 리뷰 개수 
                    for i in range(len(one_review)): 
                        # user url 
                        user_url = one_review[i].find('div', attrs = {'class':'_23Rml'}).find('a').get('href') 
                        print('user_url = '+user_url) # user url로부터 user code 뽑아내기 
                        user_code = re.findall(r"my/(\w+)", user_url)[0]
                        print('user_code = '+user_code) 
                        # review 1개에 대한 id 만들기 
                        res_code = re.findall(r"restaurant/(\d+)", thisurl)[0] 
                        review_id = str(res_code)+"_"+user_code print('review_id = '+review_id) # rating, 별점 
                        rating = one_review[i].find('span', attrs = {'class':'_2tObC'}).text
                        print('rating = '+rating) # 주의!!! 사진 리뷰 유무에 따라 날짜 파싱코드 다름 
                        # ('span', attrs = {'class':'_3WqoL'}) 
                        # 사진 없는 경우 : 총 6개 중 4번째 
                        # 사진 있는 경우 : 총 5개 중 3번째 
                        # date 
                        # 사진 리뷰 없음 
                        if len(one_review[i].find_all('span', attrs = {'class':'_3WqoL'})) == 5: date = one_review[i].find_all('span', attrs = {'class':'_3WqoL'})[2].text elif len(one_review[i].find_all('span', attrs = {'class':'_3WqoL'})) == 6: date = one_review[i].find_all('span', attrs = {'class':'_3WqoL'})[3].text else: date = "" print('date = '+date) # review 내용 try : review_content = one_review[i].find('span', attrs = {'class':'WoYOw'}).text except: # 리뷰가 없다면 review_content = "" print('리뷰 내용 : '+review_content) # image 내용 sliced_soup = one_review[i].find('div', attrs = {'class':'_1aFEL _2GO1Q'}) if (sliced_soup != None): sliced_soup = sliced_soup.find('div',attrs={'class':'dRZ2X'}) try: img_url = 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=l&size=800x800&src='+re.findall(r'src=(.*jpeg)', str(sliced_soup))[0] except : if (len(re.findall(r'src=(.*jpg)', str(sliced_soup)))!= 0): img_url = 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=l&size=800x800&src='+re.findall(r'src=(.*jpg)', str(sliced_soup))[0] elif (len(re.findall(r'src=(.*png)', str(sliced_soup)))!= 0): img_url = 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=l&size=800x800&src='+re.findall(r'src=(.*png)', str(sliced_soup))[0] else : img_url = "" else: img_url = "" print('이미지 url : '+img_url) print('----------------------------------------------') print('\n') #리뷰정보 # user_review_id user_review_id[restaurant_name][user_code] = review_id # review_json review_json[restaurant_name][review_id] = review_content # image_json image_json[restaurant_name][review_id] = img_url # rating_df_list naver_review = user_code, restaurant_name, rating, date rating_list.append(naver_review) except NoSuchElementException: none_review = "네이버 리뷰 없음" print(none_review) review_num = 0 #리뷰정보 = restaurant_name, restaurant_classification, review_num, none_review # rating_df_list naver_review = user_code, restaurant_name, none_review, none_reivew rating_list.append(naver_review) print('\n')

In [72]:
#https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=%EC%98%A4%EB%B0%B1%EB%83%A5%EB%B6%84%EC%8B%9D
url = "https://search.naver.com/search.naver?"
driver.get(url)

In [73]:
search = driver.find_element(By.ID,'nx_query')

In [74]:
#search.clear()
search.send_keys('토방닭한마리')

In [75]:
xpath = '//*[@id="nx_search_form"]/fieldset/button'
driver.find_element(By.XPATH,xpath).click()

In [77]:
xpath = '//*[@id="loc-main-section-root"]/section/div/ul/li[1]/div[2]'
driver.find_element(By.XPATH,xpath).click()

In [81]:
# <div class="flicking-camera" style="transform: translate(0px);"><a href="/restaurant/20829543/home?entry=pll&amp;from=map&amp;fromNxList=true&amp;fromPanelNum=1&amp;ts=1660786245683" role="tab" class="_2RG_o _tab-menu" aria-selected="true" title="" id="" style="width: 96px;"><span class="_3aXen">홈</span></a><a href="/restaurant/20829543/menu?entry=pll&amp;from=map&amp;fromNxList=true&amp;fromPanelNum=1&amp;ts=1660786245683" role="tab" class="_2RG_o _tab-menu" aria-selected="false" title="" id="" style="width: 96px;"><span class="_3aXen">메뉴</span></a><a href="/restaurant/20829543/review?entry=pll&amp;from=map&amp;fromNxList=true&amp;fromPanelNum=1&amp;ts=1660786245683" role="tab" class="_2RG_o _tab-menu __web-inspector-hide-shortcut__" aria-selected="false" title="" id="" style="width: 96px;"><span class="_3aXen">리뷰</span></a><a href="/restaurant/20829543/photo?entry=pll&amp;from=map&amp;fromNxList=true&amp;fromPanelNum=1&amp;ts=1660786245683" role="tab" class="_2RG_o _tab-menu" aria-selected="false" title="" id="" style="width: 96px;"><span class="_3aXen">사진</span></a></div>
#driver.find_element_by_tag_name('div').find_element_by_tag_name('a')  
driver.find_element(By.CLASS_NAME,'flicking-camera')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".flicking-camera"}
  (Session info: chrome=104.0.5112.81)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B45FD3+2187219]
	Ordinal0 [0x00ADE6D1+1763025]
	Ordinal0 [0x009F3E78+802424]
	Ordinal0 [0x00A21C10+990224]
	Ordinal0 [0x00A21EAB+990891]
	Ordinal0 [0x00A4EC92+1174674]
	Ordinal0 [0x00A3CBD4+1100756]
	Ordinal0 [0x00A4CFC2+1167298]
	Ordinal0 [0x00A3C9A6+1100198]
	Ordinal0 [0x00A16F80+946048]
	Ordinal0 [0x00A17E76+949878]
	GetHandleVerifier [0x00DE90C2+2721218]
	GetHandleVerifier [0x00DDAAF0+2662384]
	GetHandleVerifier [0x00BD137A+526458]
	GetHandleVerifier [0x00BD0416+522518]
	Ordinal0 [0x00AE4EAB+1789611]
	Ordinal0 [0x00AE97A8+1808296]
	Ordinal0 [0x00AE9895+1808533]
	Ordinal0 [0x00AF26C1+1844929]
	BaseThreadInitThunk [0x7728FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77897A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77897A6E+238]
